# Nonlinear Optimization

by Sean Lo (seanlo@mit.edu)

In [ ]:
# import Pkg;
# Pkg.add("Plots")
# Pkg.add("LinearAlgebra")
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("Random")
# Pkg.add("NLsolve")

In [ ]:
using Plots, LinearAlgebra, CSV, DataFrames, Random
using NLsolve

## Choices in step size for gradient descent

Let's investigate the impact of step size on gradient descent for different problems! The different step size choices we will look at are:
- constant step size
- exact line search, $\alpha_k = \argmin_{\alpha} f(\bm{x}^k - \alpha \bm{d}^k)$
- and backtracking line search, where you start with a "big" $\bar{\alpha}$ and decrement it until $f(\bm{x}^k - \bar{\alpha} \bm{d}^k)$ is "small enough".

---

### Exact line search v.s. constant step size, for quadratic optimization


Consider the problem 
$$\min_{x_1, x_2} f(x_1, x_2) := 5x_1^2 + x_2^2 + 4 x_1 x_2 - 14 x_1 - 6 x_2 + 20$$
This is a quadratic in $\bm{x} = (x_1, x_2)$, since we can write $f$ as:
$$f(\bm{x}) = \frac{1}{2} \bm{x}^\top  \bm{Q} \bm{x} - \bm{c}^\top \bm{x} + b$$
with $\bm{Q} = \begin{pmatrix} 10 & 4 \\ 4 & 2 \end{pmatrix}$, $\bm{c} = \begin{pmatrix} 14 \\ 6 \end{pmatrix}$, and $b = 20$.

We write functions to compute the cost $f(\bm{x})$ and gradient $\nabla f(\bm{x})$ as a function of $\bm{x}$:
\begin{align}
    \bm{d}^k = - \nabla f(\bm{x}^k) = 
    - \begin{pmatrix}
        10 x_1^k + 4 x_2^k - 14 
        \\
         2 x_2^k + 4 x_1^k - 6
    \end{pmatrix}
\end{align}

In [ ]:
function cost_function_quadratic(x::Vector)
    return 5 * x[1]^2 + x[2]^2 + 4 * x[1] * x[2] - 14 * x[1] - 6 * x[2] + 20
end

function gradient_quadratic(x::Vector)
    return [
        10 * x[1] + 4 * x[2] - 14,
         2 * x[2] + 4 * x[1] - 6,
    ]
end

We now implement gradient descent with constant step size:

In [ ]:
function gradient_descent_constant(
    cost_function::Function,
    gradient::Function,
    initial_x::Vector,  # Initial point
    alpha,              # Step size
    epsilon,            # Termination parameter
)
    # Initialization
    x = initial_x
    k = 0
    x_history = zeros(Float64, (0, 2))
    cost_history = Float64[]
    gradient_norm_history = Float64[]

    while true
        # Find descent direction d
        ## TODO: your code here
        
        # Update history
        x_history = vcat(x_history, x')
        push!(cost_history, cost_function_val)
        push!(gradient_norm_history, gradient_norm)
        # Termination condition
        ## TODO: your code here

        # Update x, increment iteration count
        ## TODO: your code here

    end
    return Dict(
        "x" => x_history,
        "cost" => cost_history,
        "gradient_norm" => gradient_norm_history,
    )
end

Similarly, we can implement gradient descent with exact line search. To do so, we should find $\alpha_k$ at each iteration depending on $\bm{x}^k$. Since we have a quadratic optimization problem, the problem $\min_{\alpha} f(\bm{x}^k + \alpha \bm{d}^k)$ has the closed-form solution:
\begin{align}
    \alpha_k = \frac{
        (d^k_1)^2 + (d^k_2)^2
    }{
        2 \Big( 5 (d^k_1)^2 + (d^k_2)^2 + 4 d^k_1 d^k_2\Big)
    }
\end{align}

In [ ]:
function gradient_descent_exact_linesearch(
    cost_function::Function,
    gradient::Function,
    initial_x::Vector,  # Initial point
    epsilon,            # Termination parameter
)
    # Initialization
    x = initial_x
    k = 0
    x_history = zeros(Float64, (0, 2))
    cost_history = Float64[]
    gradient_norm_history = Float64[]
    alpha_history = Float64[]

    while true
        # Find descent direction d
        ## TODO: your code here

        # Compute step size alpha
        ## TODO: your code here

        # Update history
        x_history = vcat(x_history, x')
        push!(cost_history, cost_function_val)
        push!(gradient_norm_history, gradient_norm)
        push!(alpha_history, alpha)

        # Termination condition
        ## TODO: your code here

        # Update x, increment iteration count
        ## TODO: your code here

    end
    return Dict(
        "x" => x_history,
        "cost" => cost_history,
        "gradient_norm" => gradient_norm_history,
        "alpha" => alpha_history,
    )
end

Let's see the results of our algorithms:

In [ ]:
initial_x = [0, 10]
epsilon = 1e-5
;

In [ ]:
results_quadratic_exact_linesearch = @time gradient_descent_exact_linesearch(
    cost_function_quadratic,
    gradient_quadratic,
    initial_x, 
    epsilon,
)

In [ ]:
results_quadratic_constant = @time gradient_descent_constant(
    cost_function_quadratic,
    gradient_quadratic,
    initial_x, 
    0.1, 
    epsilon,
)

In [ ]:
# Define the range of x and y values
x_range = -10:0.1:10
y_range = -10:0.1:10
grid = [(x, y) for x in x_range, y in y_range]

z = [cost_function_quadratic([x, y]) for (x, y) in grid]
z = reshape(z, length(x_range), length(y_range))'
;

In [ ]:
contour_quadratic_exact_linesearch = contour(
    x_range, y_range, z, 
    levels = 50, 
    c = :viridis, color = :auto, 
    legend = false,
)
Plots.plot!(
    results_quadratic_exact_linesearch["x"][:,1],
    results_quadratic_exact_linesearch["x"][:,2],
    linestyle = :dash,
    linewidth = 2,
    markershape = :circle, 
    color = :red,
    title = "Gradient descent with exact line search ($(length(results_quadratic_exact_linesearch["cost"])) iterations)"
)
Plots.savefig(contour_quadratic_exact_linesearch, "$(@__DIR__)/contour_quadratic_exact_linesearch.png")
contour_quadratic_exact_linesearch

In [ ]:
contour_quadratic_constant = contour(
    x_range, y_range, z, 
    levels = 50, 
    c = :viridis, color = :auto, 
    legend = false,
)
Plots.plot!(
    results_quadratic_constant["x"][:,1],
    results_quadratic_constant["x"][:,2],
    linestyle = :dash,
    linewidth = 2,
    markershape = :circle, 
    color = :red,
    title = "Gradient descent with constant step size ($(length(results_quadratic_constant["cost"])) iterations)"
)
Plots.savefig(contour_quadratic_constant, "$(@__DIR__)/contour_quadratic_constant.png")
contour_quadratic_constant

These results make sense! With exact line search, you make the best possible progress along one direction at each iteration, so there are less iterations. In this particular case, the optimal step size at each iteration can be computed analytically, so the work per iteration is about the same as GD with constant step size. Hence the algorithm runs much faster and consumes less memory with exact line search than with a constant step size.

However, for other functions where it's not as easy to solve the argmin problem efficiently, there is a tradeoff between doing more work per iteration and doing less iterations for exact line search.

---

### Backtracking line search v.s. constant step size, for a more complicated example

What happens when you move beyond quadratic optimization? We minimize the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function), which is often used as a test problem for optimization algorithms:
\begin{align*}   
    \min_{x_1, x_2} f(x_1, x_2) := 100(x_2 - x_1^2)^2 + (1 - x_1)^2
\end{align*}

\begin{align*}
    \nabla f(\bm{x}) = \begin{pmatrix}
        - 400 x_1 (x_2 - x_1^2) - 2 (1 - x_1)
        \\
        200 (x_2 - x_1^2)
    \end{pmatrix}
\end{align*}

Let's implement backtracking line search for this! 

In [ ]:
function cost_function_rosenbrock(x)
    return 100 * (x[2] - x[1]^2)^2 + (1 - x[1])^2
end

function gradient_rosenbrock(x)
    return [
        - 400 * x[1] * (x[2] - x[1]^2) - 2*(1 - x[1]),
          200 * (x[2] - x[1]^2),
    ]
end

In [ ]:
function gradient_descent_backtracking_linesearch(
    cost_function::Function,
    gradient::Function,
    initial_x::Vector,  # Initial point
    epsilon,            # Termination parameter
    s,                  # Initial alpha
    sigma,              # Backtracking parameter in (0, 0.5]
    beta,               # Rate of decrease in alpha in (0, 1)
)
    # Initialization
    x = initial_x
    k = 0
    x_history = zeros(Float64, (0, 2))
    cost_history = Float64[]
    gradient_norm_history = Float64[]
    alpha_history = Float64[]

    while true
        # Find descent direction d
        ## TODO: your code here

        # Compute step size alpha
        ## TODO: your code here
        
        # Update history
        x_history = vcat(x_history, x')
        push!(cost_history, cost_function_val)
        push!(gradient_norm_history, gradient_norm)
        push!(alpha_history, alpha)
        
        # Termination condition
        ## TODO: your code here

        # Update x, increment iteration count
        ## TODO: your code here

    end
    return Dict(
        "x" => x_history,
        "cost" => cost_history,
        "gradient_norm" => gradient_norm_history,
        "alpha" => alpha_history,
    )
end

In [ ]:
initial_x_rosenbrock = [0.25, 4.5]

In [ ]:
results_rosenbrock_constant = @time gradient_descent_constant(
    cost_function_rosenbrock,
    gradient_rosenbrock,
    initial_x_rosenbrock,
    0.001,
    epsilon,
);

In [ ]:
results_rosenbrock_backtracking_linesearch = @time gradient_descent_backtracking_linesearch(
    cost_function_rosenbrock,
    gradient_rosenbrock,
    # [2, 5],
    initial_x_rosenbrock,
    epsilon,
    2,      # s
    0.25,   # sigma
    0.5,    # beta
);

In [ ]:
# Define the range of x and y values
x_range = 0:0.01:3
y_range = 0:0.01:5
grid = [(x, y) for x in x_range, y in y_range]

z = [cost_function_rosenbrock([x y]) for (x, y) in grid]
z = reshape(z, length(x_range), length(y_range))'
;

In [ ]:
contour_rosenbrock_constant = contour(
    x_range, y_range, z, 
    levels = 1000, 
    c = :viridis, color = :auto, 
    legend = false,
)
Plots.plot!(
    results_rosenbrock_constant["x"][:,1],
    results_rosenbrock_constant["x"][:,2],
    linestyle = :dash,
    linewidth = 2,
    markershape = :circle, 
    color = :red,
    title = "Gradient descent with constant step size ($(length(results_rosenbrock_constant["cost"])) iterations)"
)
Plots.savefig(contour_rosenbrock_constant, "$(@__DIR__)/contour_rosenbrock_constant.png")
contour_rosenbrock_constant

In [ ]:
contour_rosenbrock_backtracking_linesearch = contour(
    x_range, y_range, z, 
    levels = 1000, 
    c = :viridis, color = :auto, 
    legend = false,
)
Plots.plot!(
    results_rosenbrock_backtracking_linesearch["x"][:,1],
    results_rosenbrock_backtracking_linesearch["x"][:,2],
    linestyle = :dash,
    linewidth = 2,
    markershape = :circle, 
    color = :red,
    title = "Gradient descent with backtracking_linesearch step size ($(length(results_rosenbrock_backtracking_linesearch["cost"])) iterations)"
)
Plots.savefig(contour_rosenbrock_backtracking_linesearch, "$(@__DIR__)/contour_rosenbrock_backtracking_linesearch.png")
contour_rosenbrock_backtracking_linesearch

Here, we see that backtracking line search is significantly faster than the naive method of GD with a constant step size! Backtracking is also a simple way to approximate exact line search, especially when we don't have a closed form expression for $\alpha_k$ in exact line search.

---

## Newton's method

Finally, let's look at how Newton's method performs on the Rosenbrock function above. Recall that Newton's method can potentially converge very quickly to local stationary points, depending on the initialization points.

So far, we've seen gradient descent on the Rosenbrock function converge to what looks like $(1, 1)$. We first use the `NLsolve.jl` package to find local minima and maxima of the Rosenbrock function, and verify that indeed $(1, 1)$ is a local minimum. Since finding stationary points of a function is equivalent to finding roots of its gradient, we use the gradient function we've defined previously:

In [ ]:
using NLsolve

In [ ]:
# Range for initial guesses
x_range = 0:0.5:3
y_range = 0:0.5:5
unique_solutions = Set{Tuple{Float64, Float64}}()
for point in Iterators.product(
    x_range,
    y_range,
)
    result = nlsolve(gradient_rosenbrock, [point[1], point[2]])
    # Convert result to tuple and round to 6 decimal places
    # (for numerical accuracy)
    solution = (
        round(result.zero[1], digits=6), 
        round(result.zero[2], digits=6),
    )
    # Add the solution to the set (only unique solutions)
    push!(unique_solutions, solution)
end

In [ ]:
unique_solutions

Hence, $(1, 1)$ is the only stationary point of the function. If we want to find out if it is a local maximum or minimum, or a saddle point, that depends on whether the eigenvalues of the Hessian at $(1, 1)$ are all negative, all positive, or neither, respectively. We first compute the Hessian:
\begin{align*}
    \nabla^2 f(\bm{x}) = \begin{pmatrix}
        - 400 (x_2 - x_1^2) + 800 x_1^2 + 2 
        & - 400 x_1
        \\
        - 400 x_1 
        & 200
    \end{pmatrix}
\end{align*}

In [ ]:
function hessian_rosenbrock(x::Vector)
    d2f_dx1dx1 = - 400 * (x[2] - x[1]^2)^2 + 800 * x[1]^2 + 2
    d2f_dx1dx2 = - 400 * x[1]
    d2f_dx2dx2 = 200
    return [d2f_dx1dx1 d2f_dx1dx2; d2f_dx1dx2 d2f_dx2dx2]
end

In [ ]:
eigenvalues = eigen(hessian_rosenbrock([1, 1])).values

Since the eigenvalues are all positive, $(1, 1)$ is indeed a local minima. (How can you show that $(1, 1)$ is a global minimum?)

Next, let's implement Newton's method:

In [ ]:
function newtons_method(
    cost_function::Function,
    gradient::Function,
    hessian::Function,
    initial_x::Vector,          # Initial point
    ;
    epsilon::Float64 = 1e-6,    # Termination parameter
    max_iters::Int = 1000,
)
    # Initialization
    x = initial_x
    k = 0
    x_history = zeros(Float64, (0, 2))
    cost_history = Float64[]

    while k < max_iters
        # Find descent direction d
        ## TODO: your code here
        # Update history
        x_history = vcat(x_history, x')
        push!(cost_history, cost_function_val)

        # Termination condition
        ## TODO: your code here

        # Update x, increment iteration count
        ## TODO: your code here

    end
    return Dict(
        "x" => x_history,
        "cost" => cost_history,
    )
end

In [ ]:
results_rosenbrock_newtons_method = newtons_method(
    cost_function_rosenbrock,
    gradient_rosenbrock,
    hessian_rosenbrock,
    initial_x_rosenbrock,
    ;
    max_iters = 100, 
)

In [ ]:
# Define the range of x and y values
x_range = 0:0.01:3
y_range = 0:0.01:5
grid = [(x, y) for x in x_range, y in y_range]

z = [cost_function_rosenbrock([x y]) for (x, y) in grid]
z = reshape(z, length(x_range), length(y_range))'

contour_rosenbrock_newtons_method = contour(
    x_range, y_range, z, 
    levels = 1000, 
    c = :viridis, color = :auto, 
    legend = false,
)
Plots.plot!(
    results_rosenbrock_newtons_method["x"][:,1],
    results_rosenbrock_newtons_method["x"][:,2],
    linestyle = :dash,
    linewidth = 2,
    markershape = :circle, 
    color = :red,
    title = "Newton's method with alpha = 1 ($(length(results_rosenbrock_newtons_method["cost"])) iterations)"
)
Plots.savefig(contour_rosenbrock_newtons_method, "$(@__DIR__)/contour_rosenbrock_newtons_method.png")
contour_rosenbrock_newtons_method

That's cool! Newton's method works very well on Rosenbrock's function. Do try other initialization points and see what happens. 